### Import Data

In [1]:
from utils import *

from IPython.display import HTML, display

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pd.options.display.float_format = '{:,.2f}'.format

In [2]:
%%time
df = csvs_to_df()

CPU times: user 52.3 s, sys: 11.6 s, total: 1min 3s
Wall time: 1min 4s


In [3]:
%%time
df_assessments = process_df(df)

CPU times: user 9.04 s, sys: 336 ms, total: 9.37 s
Wall time: 9.34 s


In [4]:
df_assessments

,distinct_id,url_name,response_id,categories,category_scores,question_labels,question_scores,total_score,timestamp,phq_suicide,phq_gad_1,phq_gad_2,phq_dep_1,phq_dep_2,wemwbs_1,wemwbs_2,wemwbs_3,wemwbs_4,wemwbs_5,wemwbs_6,wemwbs_7,wemwbs_8,wemwbs_9,wemwbs_10,wemwbs_11,wemwbs_12,wemwbs_13,wemwbs_14,retired,wsas_1,wsas_2,wsas_3,wsas_4,wsas_5,phq_gad_score,phq_dep_score,phq_mood_score,wemwbs_score,wsas_score,gs_mood_score,gs_wemwbs_score,gs_wsas_score
0,000082ab-c331-48f7-a4e3-7cd1dd7daae5,NaN,1ab911b8-f108-4a6d-93de-0f9ab5f3f36c,"[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",nan,2020-08-12 05:42:12,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,000082ab-c331-48f7-a4e3-7cd1dd7daae5,NaN,8323ac61-82a9-44b7-b3a1-052823a66223,"[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",nan,2020-08-31 10:25:12,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,000082ab-c331-48f7-a4e3-7cd1dd7daae5,NaN,72665e27-0d2e-409b-a58b-7923a431dba0,"[Humeur, Bien-être, Fonctionnement]","[8, 41, 20]","[Jamais, Plusieurs jours, Plusieurs jours, ...","[3, 2, 2, 2, 2, 4, 4, 2, 3, 2, 2, 2, 3, 3, 3, ...",72.00,2022-01-31 16:39:07,3.00,2.00,2.00,2.00,2.00,4.00,4.00,2.00,3.00,2.00,2.00,2.00,3.00,3.00,3.00,3.00,4.00,3.00,3.00,0.00,4.00,4.00,4.00,4.00,4.00,8.00,8.00,4.00,41.00,20.00,4.00,41.00,20.00
3,0000b5a9-a906-4e38-aa73-8416f6b0385a,NaN,65101fc3-bb6b-480f-aa00-c3f1e2d2f535,"[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",nan,2020-12-28 23:22:36,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,0000b5a9-a906-4e38-aa73-8416f6b0385a,NaN,26973c96-f01b-4e2c-bcf1-73cad3ded543,"[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",nan,2021-01-18 13:41:33,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376400,fffec8b3-1f66-494a-a7ce-421c982dee38,NaN,eeec9669-c4b4-44b2-96c7-e50c398cb61f,"[Mood, Well-Being, Functioning]","[5, 39, 13]","[3, 1, 2, 0, 2, 3, 3, 2, 1, 1, 3, 4, 3, 3, 3, ...","[3, 1, 2, 0, 2, 3, 3, 2, 1, 1, 3, 4, 3, 3, 3, ...",60.00,2022-01-14 22:44:30,3.00,1.00,2.00,0.00,2.00,3.00,3.00,2.00,1.00,1.00,3.00,4.00,3.00,3.00,3.00,4.00,4.00,2.00,3.00,0.00,3.00,1.00,1.00,5.00,3.00,9.00,10.00,7.00,39.00,27.00,7.00,39.00,27.00
376401,fffec8b3-1f66-494a-a7ce-421c982dee38,NaN,e24c69dd-6877-4d31-8f65-a26a03f4048b,"[Mood, Well-Being, Functioning]","[7, 43, 17]","[3, 1, 2, 2, 2, 3, 4, 1, 3, 2, 3, 4, 3, 3, 3, ...","[3, 1, 2, 2, 2, 3, 4, 1, 3, 2, 3, 4, 3, 3, 3, ...",70.00,2022-01-31 11:09:48,3.00,1.00,2.00,2.00,2.00,3.00,4.00,1.00,3.00,2.00,3.00,4.00,3.00,3.00,3.00,4.00,3.00,4.00,3.00,0.00,3.00,4.00,3.00,5.00,2.00,9.00,8.00,5.00,43.00,23.00,5.00,43.00,23.00
376402,fffec8b3-1f66-494a-a7ce-421c982dee38,NaN,499de492-7a49-4801-a748-851ce19ed5eb,"[Mood, Well-Being, Functioning]","[9, 45, 14]","[3, 2, 2, 3, 2, 4, 4, 1, 2, 2, 3, 4, 3, 3, 2, ...","[3, 2, 2, 3, 2, 4, 4, 1, 2, 2, 3, 4, 3, 3, 2, ...",71.00,2022-02-28 10:27:40,3.00,2.00,2.00,3.00,2.00,4.00,4.00,1.00,2.00,2.00,3.00,4.00,3.00,3.00,2.00,5.00,5.00,4.00,3.00,0.00,2.00,1.00,1.00,4.00,6.00,8.00,7.00,3.00,45.00,26.00,3.00,45.00,26.00
376403,ffff0ec5-b479-4d60-b16e-23267134b198,NaN,ae507450-ab55-47ca-992f-9d0be7dec065,"[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan, nan, nan,

In [5]:
# Save dataframe to csv file
df_assessments.to_csv('csvs/processed/mixpanel_assessment_complete_events_from_2021-01-01_to_2022-06-30.csv')

In [6]:
# first assessments, last assessments, and first and last for each uuid on a single line
df_assess_first = df_assessments.groupby('distinct_id', as_index=False).first()
df_assess_last = df_assessments.groupby('distinct_id', as_index=False).last()
df_assess_last = df_assess_last.add_suffix('_last')
df_assess_fl = df_assess_first.join(df_assess_last)

# add a week index
# df_assess_first['week_index'] = ((df_assess_first.timestamp - dt.datetime(2021, 1, 1)).dt.days / 7).astype(int)
# df_week = df_assess_first.groupby(df_assess_first.week_index).mean()

In [7]:
df_assess_fl['total_time'] = (df_assess_fl.timestamp_last - df_assess_fl.timestamp).dt.days
df_assess_fl['dep_diff'] = (df_assess_fl.phq_dep_score_last - df_assess_fl.phq_dep_score)
df_assess_fl['anx_diff'] = (df_assess_fl.phq_gad_score_last - df_assess_fl.phq_gad_score)
df_assess_fl['mood_diff'] = (df_assess_fl.phq_mood_score_last - df_assess_fl.phq_mood_score)
df_assess_fl['func_diff'] = (df_assess_fl.wsas_score_last - df_assess_fl.wsas_score)
df_assess_fl['wb_diff'] = (df_assess_fl.wemwbs_score_last - df_assess_fl.wemwbs_score)

dep_sd = df_assess_fl['dep_diff'].std()
anx_sd = df_assess_fl['anx_diff'].std()
mood_sd = df_assess_fl['mood_diff'].std()
func_sd = df_assess_fl['func_diff'].std()
wb_sd = df_assess_fl['wb_diff'].std()

df_assess_fl['dep_d'] = (df_assess_fl['dep_diff'] / dep_sd)
df_assess_fl['anx_d'] = (df_assess_fl['anx_diff'] / anx_sd)
df_assess_fl['mood_d'] = (df_assess_fl['mood_diff'] / mood_sd)
df_assess_fl['func_d'] = (df_assess_fl['func_diff'] / func_sd)
df_assess_fl['wb_d'] = (df_assess_fl['wb_diff'] / wb_sd)

In [8]:
df_assess_fl.to_csv('csvs/processed/mixpanel_users_assess_scores_from_2021-01-01_to_2022-06-30.csv')
# df_assess_fl = pd.read_csv('csvs/processed/mixpanel_users_assess_scores_from_2021-01-01_to_2022-06-30.csv')

In [9]:
tmp = df_assess_fl[df_assess_fl['timestamp'] >= '2022-01-01']

len(tmp[tmp.phq_mood_score <= 2])/len(tmp)

0.15456147220046984

In [10]:
df_use_resource = df[(df['event'] == 'Access Resource') | ((df['event'] == 'Page Load') & (df['url_name'] == 'resource-wrapper'))]
df_use_resource.to_csv('csvs/processed/mixpanel_use_resource_events_from_2020-04-05_to_2022-03-31.csv')

df_use_resource = pd.read_csv('csvs/processed/mixpanel_use_resource_events_from_2020-04-05_to_2022-03-31.csv')

In [11]:
#Groupby user and get the timestamps for each user
timestamps_by_user = df_use_resource.groupby(['distinct_id']).timestamp

# print(timestamps_by_user.get_group('179fbb0b04c71f-0bd657411479c4-48183301-13c680-179fbb0b04d231e'))

# Dictionary of dictionaries
# Top-level is users and then we get a count of each timestamp for each user
# This will allow us to look up a given timestamp to see if there are duplicates

timestamp_dict = {} # dict of dicts
for group in timestamps_by_user.groups: # iterate through user timestamp lists
    timestamp_dict[group] = {} # create the users dict
    group_list = timestamps_by_user.get_group(group) # get the group list
    
    for timestamp in group_list: # iterate through the timestamps
        if timestamp in timestamp_dict[group]:
            timestamp_dict[group][timestamp] += 1
        else:
            timestamp_dict[group][timestamp] = 1
            
def unique_row(row):
    if row['event'] == 'Access Resource':
        return True
    if timestamp_dict[row['distinct_id']][row['timestamp']] > 1:
        return False
    return True
       
df_accesses = df_use_resource[df_use_resource.apply(unique_row, axis=1)]

AttributeError: 'DataFrameGroupBy' object has no attribute 'timestamp'

In [ ]:
df_accesses.to_csv('csvs/processed/mixpanel_use_resource_unduplicated_events_from_2020-04-05_to_2022-03-31.csv')
# df_accesses = pd.read_csv('csvs/processed/mixpanel_use_resource_unduplicated_events_from_2020-04-05_to_2022-02-28.csv')

In [ ]:
### %s of users who took assessments and accessed resources

df_first_access = \
df_accesses.groupby('distinct_id').first()
df_first_access.rename(columns = {'timestamp': 'timestamp_access'}, inplace=True)

df_first_access_merged = \
    df_first_access.merge(
        get_grouped_event(df, 
            'Signup', 'distinct_id', function='first', 
            rename={'timestamp': 'timestamp_signup'})[['distinct_id', 'timestamp_signup']], 
        on='distinct_id',
        how="outer")

df_first_access_merged = \
    df_first_access_merged.merge(
        get_grouped_event(df, 
            'Self Assessment Completed', 'distinct_id', function='first', 
            rename={'timestamp': 'timestamp_assess'})[['distinct_id', 'timestamp_assess']], 
        on='distinct_id',
        how="left")

df_first_access_merged = \
    df_first_access_merged.merge(
        get_grouped_event(df, 
            'Complete Assessment', 'distinct_id', function='count', 
            rename={'timestamp': 'assess_count'})[['distinct_id', 'assess_count']], 
        on='distinct_id',
        how="left")

df_first_access_merged = \
    df_first_access_merged.merge(
        get_grouped_event(df, 
            'Page Load', 'distinct_id', function='last', 
            rename={'language': 'language_last'})[['distinct_id', 'language_last']], 
        on='distinct_id',
        how="left"
    )

df_first_access_merged[['timestamp_signup', 'timestamp_access', 'timestamp_assess']] = \
df_first_access_merged[['timestamp_signup', 'timestamp_access', 'timestamp_assess']].apply(pd.to_datetime, errors='coerce')

df_first_access_merged['time_to_access'] = df_first_access_merged['timestamp_access'] - df_first_access_merged['timestamp_signup'] 
df_first_access_merged['time_to_access_30'] = df_first_access_merged['time_to_access'] < dt.timedelta(days=30)

df_access_count = df_accesses.groupby('distinct_id', as_index=False).count()[['distinct_id', 'event']]
df_access_count.rename(columns = {'event': 'access_count'}, inplace=True)

df_first_access_merged = \
    df_first_access_merged.merge(
        df_access_count, on='distinct_id', how="outer"
    )

print('accessed resource: ', df_first_access_merged['timestamp_access'].count())
print('accessed in 30   : ', df_first_access_merged['time_to_access_30'].sum())
print('initial assess   : ', df_first_access_merged['timestamp_assess'].count())
print('assess > 1       : ', (df_first_access_merged['assess_count'] > 1).sum())
print('access > 0       : ', (df_first_access_merged['access_count'] > 0).sum())

In [ ]:
df_access_count = \
df_access_count = df_accesses.groupby('distinct_id', as_index=False).count()[['distinct_id', 'event']]
df_access_count.rename(columns = {'event': 'access_count'}, inplace=True)

df_assess_fl_access = df_assess_fl.merge(df_access_count, on='distinct_id',how='left')
df_assess_fl_access['access_count'] = df_assess_fl_access['access_count'].fillna(0)

In [ ]:
# df_assess_fl_access.to_csv('csvs/processed/mixpanel_users_assessments_from_2021-01-01_to_2022-03-31.csv')
df_assess_fl_access = pd.read_csv('csvs/processed/mixpanel_users_assessments_from_2021-01-01_to_2022-03-31.csv')

In [ ]:
len(df_assess_fl_access[df_assess_fl_access['timestamp'] > '2021-07-01'])

In [ ]:
id_age_df = csv_aggregator('age_range')
id_age_df = id_age_df[['id', 'age_range']]
id_age_df.columns = ['distinct_id', 'age_range']
print(len(id_age_df))
id_age_df.drop_duplicates(inplace=True)
print(len(id_age_df))

In [ ]:
df.device[0:10]

In [ ]:
# temp = df_assess_fl_access.merge(id_age_df, on="distinct_id", how="left")
print(len(temp))
# temp2 = \
#     temp.merge(
#         get_grouped_event(df, 
#             'Page Load', 'distinct_id', function='last', 
#             rename={'language': 'language_last'})[['distinct_id', 'language_last']], 
#         on='distinct_id',
#         how="left"
#     )
print(len(temp2))
temp3 = \
    temp.merge(
        get_grouped_event(df, 
            'Page Load', 'distinct_id', function='last', 
            rename={'language': 'language_last'})[['distinct_id', 'language_last']], 
        on='distinct_id',
        how="left"
    )


In [ ]:
df_first_access_merged.to_csv('csvs/processed/mixpanel_users_outerjoin_from_2021-01-01_to_2022-02-28.csv')

In [ ]:
# df_assess_fl_access.to_csv('csvs/processed/mixpanel_users_assessments_left_join_from_2021-01-01_to_2022-02-28.csv')
# df_assess_fl_access = pd.read_csv('csvs/processed/mixpanel_users_assessments_left_join_from_2021-01-01_to_2022-02-28.csv')
# temp2.to_csv('csvs/processed/mixpanel_users_assessments_left_join_from_2021-01-01_to_2022-02-28.csv')
df_assess_fl_access = pd.read_csv('csvs/processed/mixpanel_users_assessments_left_join_from_2021-01-01_to_2022-02-28.csv')

In [ ]:
df_assess_fl_access.columns

In [ ]:
def get_results_table (df, headers, score_vars, score_labels, std=True, d=False):
    
    # check for correct number of labels problems
    if len(score_vars) != len(score_labels): 
        raise Exception("score_vars and score_labels must have equal length")
    
    # set up our list of tuples from score_vars and score_labels
    tup_list = []
    for i in range(0, len(score_vars)):
        tup_list.append((score_vars[i], score_labels[i]))

    # create our tables based on what vars indicated
    table = []
    for tup in tup_list:
        label = tup[1]
        mean = df[[tup[0]]].mean()
        stdev = df[[tup[0]]].std()
        
        add_list = [label, mean]
        if std:
            add_list.append(stdev)
        if d:
            add_list.append(mean/stdev)
            
        table.append(add_list)
        
    # make sure headers is the right length
    if len(headers) != len(add_list): 
        raise Exception("incorrect number of headers")
    
        
    text_output = tabulate.tabulate(table, headers=headers, tablefmt='html', floatfmt=["",".2f",".2f",".2f",".0%",".2f",".3f"])

    display(text_output)

In [ ]:
len(df_assess_fl[df_assess_fl['timestamp'] >= '2022-01-01'])

In [ ]:
df_14 = df_assess_fl_access[df_assess_fl_access['total_time'] >= 14]
df_14 = df_14[df_14['timestamp'] <= '2022-03-31']
df_14 = df_14[df_14['timestamp'] >= '2021-07-01']
print(len(df_14))
df_14 = df_14[df_14['access_count'] >= 1]
print(len(df_14))

# ax = df_14_access['total_time'].plot(kind='hist', bins=15)
# ax.set_xlabel('TOTAL_TIME')
# ax.set_ylabel('Frequency')
# plt.show()


headers = ['var', 'diff', 'sd', 'd']
score_vars = ['mood_diff', 'dep_diff', 'anx_diff', 'wb_diff', 'func_diff']
score_labels = ['mood', 'dep', 'anx', 'wb', 'func']

# get_results_table(df_14_access, headers, score_vars, score_labels, d=True)
get_results_table(df_14, headers, score_vars, score_labels, d=True)

In [ ]:
x = stats.zscore(df_14.access_count)

y = stats.zscore(df_14.mood_diff)
model_mood = sm.OLS(y, x , missing='drop').fit()

corr_mood = df_14[['mood_diff','access_count']].corr()

y = df_14.dep_diff
model_dep = sm.OLS(y, x , missing='drop').fit()

y = df_14.anx_diff
model_anx = sm.OLS(y, x , missing='drop').fit()

y = stats.zscore(df_14.wb_diff)
model_wb = sm.OLS(y,x , missing='drop').fit()

corr_wb = df_14[['wb_diff','access_count']].corr()

y = stats.zscore(df_14.func_diff)
model_func = sm.OLS(y,x , missing='drop').fit()

corr_func = df_14[['func_diff','access_count']].corr()

print(model_moodz.summary())
print(corr_mood)
print(model_mood.summary())
print(model_dep.summary())
print(model_anx.summary())
print(model_wb.summary())
print(corr_wb)
print(model_func.summary())
print(corr_func)

In [ ]:
df_14.access_count

df_14[['access_count', 'wb_diff']].corr()